# ViralCutter
Uma alternativa gratuita ao `opus.pro` e ao `vidyo.ai`

# Suporte em:
[![](https://dcbadge.limes.pink/api/server/tAdPHFAbud)](https://discord.gg/tAdPHFAbud)

# TODO📝
- [x] Release code
- [ ] Huggingface SpaceDemo
- [x] Two face in the cut
- [x] Custom caption and burn
- [x] Make the code faster
- [ ] More types of framing beyond 9:16

In [1]:
#@title 🛠️ Instalação
import os
import subprocess
import shutil
from IPython.display import clear_output

# 1. Limpeza TOTAL
print("🧹 Limpando instalação anterior...")
%cd /content
if os.path.exists("ViralCutter"):
    shutil.rmtree("ViralCutter")

!git clone -b dev https://github.com/RafaelGodoyEbert/ViralCutter.git
%cd /content/ViralCutter

print("⏳ Instalando gerenciador UV e drivers do sistema...")

# 2. Instalar UV e Drivers Linux
subprocess.run(['pip', 'install', 'uv'], check=True)
subprocess.run('sudo apt update -y && sudo apt install -y libcudnn8 ffmpeg xvfb', shell=True, check=True)

# 3. Criar Ambiente Virtual
print("⏳ Criando ambiente virtual...")
subprocess.run(['uv', 'venv', '.venv'], check=True)

# 4. INSTALAÇÃO DAS DEPENDÊNCIAS
print("⏳ Instalando Bibliotecas...")

# Passo A: WhisperX e Requisitos Básicos (Deixe instalar o que quiserem)
cmds_fase_1 = [
    "uv pip install --python .venv git+https://github.com/m-bain/whisperx.git",
    "uv pip install --python .venv -r requirements.txt",
    "uv pip install --python .venv yt-dlp pytubefix",
    "uv pip install yt_dlp"
]

for cmd in cmds_fase_1:
    subprocess.run(cmd, shell=True, check=True)

# Passo B: CORREÇÃO DE ALINHAMENTO E GEMINI
# - google-generativeai: Para o Gemini funcionar
# - pandas: Para separar palavras
# - transformers==4.46.3: VERSÃO CRÍTICA. Versões mais novas exigem Torch 2.6 e quebram o alinhamento.
# - accelerate: Ajuda no carregamento do modelo
print("🔨 Aplicando downgrade estratégico no Transformers...")
extra_libs = [
    "uv pip install --python .venv google-generativeai",
    "uv pip install --python .venv pandas",
    "uv pip install --python .venv onnxruntime-gpu",
    "uv pip install --python .venv transformers==4.46.3 accelerate>=0.26.0"
]

for cmd in extra_libs:
    subprocess.run(cmd, shell=True, check=True)

# Passo C: O MARTELO FINAL (Torch 2.3.1 Estável)
# Reinstalamos por último para garantir que nada atualizou ele sem querer
print("🔨 Forçando versão estável do Torch (2.3.1)...")
cmd_fix_torch = (
    "uv pip install --python .venv "
    "torch==2.3.1+cu121 torchvision==0.18.1+cu121 torchaudio==2.3.1+cu121 "
    "--index-url https://download.pytorch.org/whl/cu121"
)
subprocess.run(cmd_fix_torch, shell=True, check=True)

# Passo D: Trava do Numpy
print("🔨 Travando Numpy...")
subprocess.run("uv pip install --python .venv 'numpy<2.0' setuptools==69.5.1", shell=True, check=True)

# Passo E: Correção de Visão Computacional (CRÍTICO)
print("🔨 Corrigindo InsightFace e MediaPipe...")

# 1. Garante que o InsightFace tenha o motor necessário (evita que ele falhe)
subprocess.run("uv pip install --python .venv insightface onnxruntime-gpu", shell=True, check=True)

# 2. Limpeza Radical e Reinstalação do MediaPipe
# Remove versões quebradas
subprocess.run("uv pip uninstall --python .venv mediapipe protobuf flatbuffers", shell=True)

# 3. Instala versão FLEXÍVEL respeitando limites críticos
# mediapipe>=0.10.0: Pega a versão mais recente compatível com Py3.12
# protobuf>=3.20,<5.0: Garante compatibilidade sem quebrar o sistema antigo
subprocess.run("uv pip install --python .venv 'mediapipe>=0.10.0' 'protobuf>=3.20,<5.0' 'flatbuffers>=2.0'", shell=True, check=True)

# 5. Configurar Monitor
os.system('Xvfb :1 -screen 0 2560x1440x8 &')
os.environ['DISPLAY'] = ':1.0'

clear_output()
print("✅ Instalação V7 Finalizada!")
print("- Transformers 4.46.3 (Compatível com Alinhamento): INSTALADO")
print("- Torch 2.3.1: ATIVO")

✅ Instalação V7 Finalizada!
- Transformers 4.46.3 (Compatível com Alinhamento): INSTALADO
- Torch 2.3.1: ATIVO


In [ ]:
#@title 🚀 Configuração e Execução
import os

%cd /content/ViralCutter

# 1. Configura o "Display" falso (para evitar erros de interface gráfica)
os.system('Xvfb :1 -screen 0 2560x1440x8 &')
os.environ['DISPLAY'] = ':1.0'

# 2. CORREÇÃO DO ERRO ATUAL:
# Forçamos o Matplotlib a usar o backend 'Agg' (modo sem cabeça/headless)
# Isso ignora o 'module://matplotlib_inline.backend_inline' que o Colab força e quebra o script.
os.environ['MPLBACKEND'] = 'Agg'

print("🚀 Iniciando ViralCutter usando o ambiente virtual...")
print("⚠️ Ignore os avisos de 'UserWarning', aguarde o link public URL.")

# Executa usando o Python do ambiente virtual
!/content/ViralCutter/.venv/bin/python webui/app.py --colab

/content/ViralCutter
🚀 Iniciando ViralCutter usando o ambiente virtual...
⚠️ Ignore os avisos de 'UserWarning', aguarde o link public URL.
/content/ViralCutter/webui/app.py:204: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme, css. Please pass these parameters to launch() instead.
  with gr.Blocks(title=i18n("ViralCutter WebUI"), theme=gr.themes.Default(primary_hue="blue", neutral_hue="slate"), css=css) as demo:
Running in Colab mode. Generating public link...
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://7f3b04ec670707105b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#Créditos

Inspirado no [reels clips automator](https://github.com/eddieoz/reels-clips-automator) e no [YoutubeVideoToAIPoweredShorts](https://github.com/Fitsbit/YoutubeVideoToAIPoweredShorts)<br>

---
![Rafa.png](https://i.imgur.com/cGknQpU.png;base64)

Desenvolvido por **Rafa.Godoy**<br>
[ ![GitHub](https://img.shields.io/badge/github-%23121011.svg?style=for-the-badge&logo=github&logoColor=white) ](https://github.com/rafaelGodoyEbert)<br>
[ ![X](https://img.shields.io/twitter/url?url=https%3A%2F%2Ftwitter.com%2FGodoyEbert) ](https://twitter.com/GodoyEbert)<br>
[Instagram](https://www.instagram.com/rafael.godoy.ebert/)<br>
[ ![](https://dcbadge.vercel.app/api/server/aihubbrasil) ](https://discord.gg/aihubbrasil)

`0.1v Alpha`<br>

Apenas uma alternativa gratuita ao `opus.pro` e ao `vidyo.ai`<br>
